First of all we need all the necessary librarys to run this beauty!

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from random import randrange, shuffle
from os import path
import librosa
from librosa.display import specshow
import IPython.display as ipd
from math import floor
import time

***Check to see if GPU is available

In [2]:
if (len(tf.config.experimental.list_physical_devices("GPU")) > 0):
    print("GPU is available!")
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.75)
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

GPU is available!


Open and read the csv with all the audio information

In [3]:
csvPath    = r"F:\labeled_audio_data.csv"
specPath   = r"C:\Users\cboen\Documents\Programmierungen\AudioDataScience\Labeled_Specs"
audioPath  = r"F:\Labeled_Audio"
modelPath  = r"F:\Models"
logPath    = r"F:\Logs"

version      = r"2.9.2" #When you make changes, update the version!!
creationTime = time.strftime("%d.%m.%H.%M.%S", time.localtime())

dataFrame = pd.read_csv(csvPath)
dataDictionary = dataFrame.to_dict()
print(dataDictionary.keys())

dict_keys(['Unnamed: 0', 'filename', 'mainSound', 'length', 'sampleRate', 'quality', 'isCut', 'isMixed', 'isChecked', 'threat', 'salience', 'importance'])


Now just use a snippet of the data for now,
This is to be RAM friendly and for testing purposes.
The Training and Test Data will be a list containing a list that looks like the following:
(filename, mainsound, spectrogram, threat, salience, importance)
filename is a string of the filename
mainsound is a string of the sound category
Spectrogram holds a numpy array of the log spectrogram
threat, salience and importance are single numbers and the output values

In [ ]:
numAudiosPerBatch = 2200 #Number of audiofiles to train with per Batch

allFileNames = list(dataDictionary['filename'].values())
#allMainSound = list(dataDictionary['mainSound'].values())
#allThreat = list(dataDictionary['threat'].values())
#allSalience = list(dataDictionary['salience'].values())
#allImportance = list(dataDictionary['importance'].values())

training_data = []

dataIndex = 0
numBatches   = floor(len(allFileNames) / numAudiosPerBatch)
testDataSize = len(allFileNames) % numAudiosPerBatch
print("TrainBatches:", numBatches, ", Testdata:", testDataSize)

ids = list(range(0, len(allFileNames)))
shuffle(ids) #, int(time.strftime("%S", time.localtime()))/60

def createTrainingData():
    #ID list ro iterate through all the data
    global dataIndex
    print("Get new training data: ", int(dataIndex/numAudiosPerBatch), "/", (numBatches-1), sep = "")
    for i in range(numAudiosPerBatch):
        perc = int( 20 * i / numAudiosPerBatch)
        bar = "[" + "=" * (perc) + ">" + "-" * (19-perc) + "]"
        print(bar, end="\r")
        audioFile = ids[i+dataIndex]
        filename = allFileNames[audioFile]
        #Load the spectrogram
        try:
            specFilePath = path.join(specPath, (filename+".npz"))
            spectrogramNP = np.load(specFilePath)
            spectrogram = spectrogramNP[spectrogramNP.files[0]]
            spectrogram = spectrogram[0]
            #Load the target values
            mainSound  = dataDictionary['mainSound'][audioFile]
            targetValue = [0, 0]
            if mainSound == "siren" or mainSound == "emergency_vehicle":
                targetValue[0] = 1
            else:
                targetValue[1] = 1
            training_data.append([filename, mainSound, spectrogram, targetValue])
        except Exception as e:
            #print(e)
            pass
    dataIndex += numAudiosPerBatch
    print()
    print("Done!")

test_data = []

def createTestData():
    for i in range(len(allFileNames)-testDataSize, len(allFileNames)):
        audioFile = ids[i]
        filename = allFileNames[audioFile]
        #Load the spectrogram
        try:
            specFilePath = path.join(specPath, (filename+".npz"))
            spectrogramNP = np.load(specFilePath)
            spectrogram = spectrogramNP[spectrogramNP.files[0]]
            spectrogram = spectrogram[0]
            #Load the target values
            mainSound  = dataDictionary['mainSound'][audioFile]
            targetValue = [0, 0]
            if mainSound == "siren" or mainSound == "emergency_vehicle":
                targetValue[0] = 1
            else:
                targetValue[1] = 1
        
            test_data.append([filename, mainSound, spectrogram, targetValue])
        except:
            pass

createTrainingData()
createTestData()
spec_width  = len(training_data[0][2])
spec_height = len(training_data[0][2][0])
#print(spec_width, spec_height, sep="|") #spectrogram size:

#del allMainSound, allThreat, allSalience, allImportance, ids

TrainBatches: 1 , Testdata: 91
Get new training data: 0/0


***Lets plot a random spectrogram!

In [ ]:
audioToPick = randrange(0,len(training_data),1)
print(training_data[audioToPick][0], " : ",  training_data[audioToPick][1])
#plt.imshow(training_data[audioToPick][2], cmap="plasma", aspect="auto")
fig, ax = plt.subplots()
img = specshow(training_data[audioToPick][2], y_axis='log', x_axis='time', ax=ax)
ax.set_title('Power spectrogram')
fig.colorbar(img, ax=ax, format="%+2.0f dB")
ipd.Audio(path.join(audioPath, (training_data[audioToPick][0]+".wav")))

***Lets describe the dataset:

In [ ]:
def describeTrainingData():
    print("Number of audiofiles:", len(training_data))
    print("Lets get the percentage of each Class:")
    categorys = set(dataDictionary['mainSound'].values())
    trainingDataCategorys = [data[1] for data in training_data] 
    for key in categorys:
        print("Key:", key.ljust(17, " "), ", perc.:", round(trainingDataCategorys.count(key)/len(training_data), 3))
    del categorys, trainingDataCategorys

describeTrainingData()

In [ ]:
train_x=[]
train_y=[]
def sortTrainingData():
    global train_x, train_y
    train_x = [data[2] for data in training_data]
    #train_y = [[data[3], data[4], data[5]] for data in training_data]
    train_y = [data[-1] for data in training_data]
    train_x = np.array(train_x).reshape(-1, spec_width, spec_height, 1)
    train_y = np.array(train_y)

sortTrainingData()

Now the really fun part!
Create a Convolutional network that can get one spectrogram as input data and outputs three levels for threat, salience and importance!

In [ ]:
model = keras.models.Sequential([
    #Input layer
    keras.layers.Conv2D(64, (3,3), strides=(1,1), padding="valid", activation="relu", input_shape=train_x.shape[1:]),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, (3,3), strides=(1,1), activation="relu"),
    keras.layers.MaxPool2D((2,2)),
    #...repeat until happy...
    
    #Flatten layer: make 1D out of 2D
    keras.layers.Flatten(),
    
    #Hidden layer(s) 
    #keras.layers.Dense(64, activation = 'relu'),
    #keras.layers.Dense(32, activation = 'gelu'),
    keras.layers.Dense(32, activation = 'sigmoid'),
    
    #output layer
    keras.layers.Dense(len(train_y[0]), activation = 'softmax'),
])

modelName = "2Conv3Dense-{}-".format(creationTime) + version
print("Name:", modelName)
print(model.summary())

#optimizer = keras.optimizers.Adam(learning_rate = 0.01)
tensorboard = tf.keras.callbacks.TensorBoard(path.join(logPath, modelName), histogram_freq=1)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Now lets get the first training session done!

In [ ]:
for batch in range((numBatches-1)):
    try:
        model.fit(train_x, train_y, validation_split=0.1,batch_size=20, epochs=7, callbacks=[tensorboard])
    except Exception as e:
        print("Something went wrong!")
        print(e)
    training_data = []
    createTrainingData()
    sortTrainingData()

try:
    model.fit(train_x, train_y, validation_split=0.1,batch_size=20, epochs=7, callbacks=[tensorboard])
except Exception as e:
    print("Something went wrong!")
    print(e)

***Lets let the model predict some data!

In [13]:
i = randrange(dataIndex, len(allFileNames), 1)
audioFile = ids[i]
filename = allFileNames[audioFile]
specFilePath = path.join(specPath, (filename+".npz"))
spectrogramNP = np.load(specFilePath)
spectrogram = spectrogramNP[spectrogramNP.files[0]]
spectrogram = spectrogram[0]
test_x = np.array(spectrogram).reshape(-1, spec_width, spec_height, 1)
mainSound = dataDictionary['mainSound'][ids[i]]
targetValue = [0, 0]
if mainSound == "siren" or mainSound == "emergency_vehicle":
    targetValue[0] = 1
else:
    targetValue[1] = 1
test_y = np.array(targetValue)
print("Infos: ", end="")
print( filename, dataDictionary['mainSound'][ids[i]], sep=" | ")
#print(path.join(audioPath, (filename+".wav")))
print("Goal:  ", test_y)
prediction = model(test_x)
print("Result: ", end="")
print([round(val, 2) for val in prediction.numpy()[0]])
#print("max Val:", np.argmax(prediction.numpy()[0]))
ipd.Audio(path.join(audioPath, (filename+".wav")))

Infos: 55018-0-0-37 | interior
Goal:   [0 1]
Result: [0.45, 0.55]


In [29]:
training_data = test_data[:]
sortTrainingData()
predict_y = model.predict(train_x)

***Get total errors of the test dataset

Last but not least, lets save the model.

In [37]:
model.evaluate(train_x, train_y)

17/17 [==============================] - 1s 30ms/step - loss: 0.4476 - accuracy: 0.5829


[0.447644978761673, 0.5828678011894226]

In [14]:
model.save(path.join(modelPath, modelName))

INFO:tensorflow:Assets written to: F:\Models\1Conv2Dense-13.03.14.04.28-1.2\assets
